In [ ]:
!pip install -q -r requirements.txt

In [ ]:
# Cell 1: Import necessary libraries
import os
import requests
import time
import csv
import sys
import pandas as pd
import requests
import openai
import datetime
#import markdown

In [ ]:
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    print(key)
    #print(value)
    os.environ[key] = value

In [ ]:
# Cell 2: Define the sentiment analysis function
def analyze_sentiment(json_response, target_ticker):
    print(f"Starting sentiment analysis for ticker: {target_ticker}")
    
    if not json_response or "feed" not in json_response:
        print("No valid data in JSON response.")
        sys.exit(1)
        return "No data available for analysis"

    sentiment_label = "Neutral"  # Default sentiment if no stronger signals found
    highest_relevance = 0  # Track the highest relevance score

    for item in json_response.get("feed", []):
        for ticker_data in item.get("ticker_sentiment", []):
            if ticker_data["ticker"] == "FOREX:"+str(target_ticker):
                relevance_score = float(ticker_data.get("relevance_score", 0))
                sentiment_score = float(ticker_data.get("ticker_sentiment_score", 0))
                
                if relevance_score > highest_relevance:
                    highest_relevance = relevance_score
                    if sentiment_score <= -0.35:
                        sentiment_label = "Bearish"
                    elif -0.35 < sentiment_score <= -0.15:
                        sentiment_label = "Somewhat-Bearish"
                    elif -0.15 < sentiment_score < 0.15:
                        sentiment_label = "Neutral"
                    elif 0.15 <= sentiment_score < 0.35:
                        sentiment_label = "Somewhat_Bullish"
                    elif sentiment_score >= 0.35:
                        sentiment_label = "Bullish"
                    print(f"Updated sentiment label to {sentiment_label} based on relevance and score")

    print(f"Final sentiment label for {target_ticker}: {sentiment_label}")
    return sentiment_label

In [ ]:
# Cell 3: Define the function to fetch sentiment data
def fetch_sentiment_data(api_endpoint, ticker, api_key, sort='LATEST', limit=50):
    params = {
        'function': 'NEWS_SENTIMENT',
        'tickers': ticker,
        'apikey': api_key,
        'sort': sort,
        'limit': limit
    }
    response = requests.get(api_endpoint, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error fetching data: {response.status_code}"


In [ ]:
# Cell 4: Set API endpoint and key, and define tickers
api_endpoint = "https://www.alphavantage.co/query"
api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")
#print(api_key)
tickers = ["EUR", "USD", "JPY", "GBP", "CAD", "AUD", "CHF", "NZD", "RUB"]


In [ ]:
# Cell 5: Fetch sentiment data and analyze it
for base_ticker in tickers:
    print(f"Fetching sentiment data for: {base_ticker}")
    json_response = fetch_sentiment_data(api_endpoint, "FOREX:"+base_ticker, api_key)
    if json_response is None:
        print("Failed to fetch sentiment data.")
    base_sentiment = analyze_sentiment(json_response, base_ticker)
    with open('sentiment_data.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([base_ticker, base_sentiment])
        print(f"Sentiment for {base_ticker}: {base_sentiment}")
        print("Sentiment data saved to CSV file.")
    time.sleep(45)

In [ ]:
# Read data from CSV file
df = pd.read_csv('sentiment_data.csv', header=None, names=['Currency', 'Sentiment'])

# Create DataFrame from sample data
df = pd.DataFrame(df)

In [ ]:
# Corrected function to determine the action
def determine_action(sentiment1, sentiment2):
    # Define the strength of sentiment
    sentiment_strength = {
        'Bearish': 1, 
        'Somewhat-Bearish': 2, 
        'Somewhat_Bearish': 2, 
        'Neutral': 3, 
        'Somewhat_Bullish': 4, 
        'Somewhat_Bullish': 4, 
        'Bullish': 5
    }
    
    # Determine the action based on the relative strength of sentiment
    if sentiment_strength[sentiment1] > sentiment_strength[sentiment2]:
        return "Buy"
    elif sentiment_strength[sentiment1] < sentiment_strength[sentiment2]:
        return "Sell"
    else:
        return "Neutral"

In [ ]:
# Create list to hold data
pair_data = []

# Populate list with pair data
for index, row in df.iterrows():
    for index2, row2 in df.iterrows():
        if row['Currency'] != row2['Currency']:
            action = determine_action(row['Sentiment'], row2['Sentiment'])
            pair_data.append({
                "Currency 1": row['Currency'],
                "Sentiment 1": row['Sentiment'],
                "Currency 2": row2['Currency'],
                "Sentiment 2": row2['Sentiment'],
                "Action": action
            })


In [ ]:

# Create DataFrame from list
result_df = pd.DataFrame(pair_data)

# Add columns based on actions
result_df['Consider Trading Now'] = result_df['Action'].apply(lambda x: 'Yes' if x in ['Buy', 'Sell'] else 'No')
result_df['Consider Buy Now'] = result_df['Action'].apply(lambda x: 'Yes' if x == 'Buy' else 'No')
result_df['Consider Sell Now'] = result_df['Action'].apply(lambda x: 'Yes' if x == 'Sell' else 'No')

# Export to CSV
result_df.to_csv('output.csv', index=False)

In [ ]:
# Function to generate report using OpenAI SDK
def generate_report(dataframe):
    print("Generating report...")
    api_key = os.getenv('DEEPSEEK_API_KEY')
    if not api_key:
        raise ValueError("DEEPSEEK_API_KEY environment variable is not set")
    
    #print("API KEY::", api_key)

    client = openai.OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

    now = datetime.datetime.now(datetime.timezone.utc)
    market_time = now.strftime('%H:%M UTC')
    
    # Determine the active market based on the current UTC time
    if 21 <= now.hour < 6:  # Sydney session: 9:00 pm to 6:00 am UTC
        active_market = "Sydney"
    elif 0 <= now.hour < 9:  # Tokyo session: 12:00 am to 9:00 am UTC
        active_market = "Tokyo"
    elif 7 <= now.hour < 16:  # London session: 7:00 am to 4:00 pm UTC
        active_market = "London"
    elif 13 <= now.hour < 22:  # New York session: 1:00 pm to 10:00 pm UTC
        active_market = "New York"
    else:
        active_market = "Other"

    prompt = (f"Assume the role of a forex financial analyst providing a report of trade recommendations for {now.strftime('%Y-%m-%d %H:%M')} "
              f"based off news sentiment. Consider the market time {market_time} and the active market ({active_market}) when formulating your recommendations.")
    messages = [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt + "\n\n" + dataframe.to_string()}
    ]

    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
            stream=False
        )
        return response.choices[0].message.content
    except openai.AuthenticationError as e:
        print(f"Authentication error: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example DataFrame
data = {
    'Date': ['2023-04-01', '2023-04-02', '2023-04-03'],
    'News Sentiment': ['Positive', 'Negative', 'Neutral']
}
result_df = pd.DataFrame(data)

# Generate report
report = generate_report(result_df)

# Save or print the report
if report:
    with open('report.txt', 'w') as file:
        file.write(report)
else:
    print("Failed to generate report.")

In [ ]:
print("Report generated and saved as 'report.txt'")
Markdown(report)
print("Output data saved as 'output.csv'")
print(result_df)